In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [ ]:
# requesting site content + print response

from requests import get
url = 'https://libraries.ucsd.edu/farmworkermovement/gallery/thumbnails.php?album=519&page=1'
response = get(url)
print(response.text[:500])

In [ ]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

In [ ]:
# find + print photo info containers

photo_containers = html_soup.find_all('td', class_ = 'thumbnails')
print(type(photo_containers))
print(len(photo_containers))

In [ ]:
# get info from first photo

first_photo = photo_containers[0]
first_photo

In [ ]:
# returns img number

first_number = first_photo.find('span', class_='thumb_title')
first_number

In [ ]:
# returns img info

first_photo.img

In [ ]:
# returns img title info

fimage_metadata = first_photo.find('img', class_= 'image')['title']
fimage_metadata

In [ ]:
metadata_info_clean = fimage_metadata.replace("\r\n",",").split(",")
metadata_info_clean

In [ ]:
# returns caption info w/ tag

first_caption = first_photo.find('span', class_='thumb_caption')
first_caption

In [ ]:
# returns caption info w/o tag

first_caption.text

In [47]:
# full working code
# be sure to install openpyxl to export as excel file

from bs4 import BeautifulSoup
import pandas as pd
import requests
import re

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36'}

photoinfo = [ ]

def getMetadata(page):
    url = f'https://libraries.ucsd.edu/farmworkermovement/gallery/thumbnails.php?album=546&page={page}' # change out album number
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    photo = soup.find_all('td', {'class': 'thumbnails'})
    
    for container in photo:
        metadata = {
        # for photo number
        # NOTE: some galleries include "Photo Title" instead of "Photo Number"; class is same
        'Photo Number': container.find('span', {'class': 'thumb_title'}),
        # for photo info
        'Photo Metadata': [img['title'] for img in container.find_all('img', class_='image')],        
        # for photo caption
        'Photo Caption': container.find('span', {'class': 'thumb_caption'}),
        }
        photoinfo.append(metadata)

for x in range(1,50): # adjust range to number of pages available in each gallery OR set large enough number to capture all pages (takes longer to process)
    getMetadata(x)

df = pd.DataFrame(photoinfo)
df.to_excel('honorstrike.xlsx') # update file name
print('Done!') 

Done!


Excel file is saved in the same location as notebook. Some additional cleaning must be done in Excel file.

**To remove ```<span>``` tag (Photo Number & Photo Caption columns):** select column > Find & Replace (CTRL/CMD + H) > in 'Find' field, enter ```<*>```, leave 'Replace' field blank > click 'Replace All'

**To remove ```\r\n``` & separate with comma (Photo Metadata column):** select column > Find & Replace (CTRL/CMD + H) > in 'Find' field, enter ```\r\n```, in 'Replace' field, enter ```, ``` (add space after comma) > click 'Replace All'